In [1]:
import pandas
import datetime
import numpy
import scipy.optimize
from scipy.stats import linregress
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import time
import bs4
import urllib
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib.ticker import FuncFormatter, MaxNLocator
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
# Some constants
cutOffMLTMLATUniqCnt = 4

In [3]:
asyDF = pandas.read_csv( "../data/Asy_processed.txt", sep=' ' )
asyDF["date"] = pandas.to_datetime(asyDF["datetimeStr"], format='%Y%m%d-%H-%M')
asyDF["hour"] = asyDF["date"].apply(lambda x: x.strftime('%H'))
asyDF["minute"] = asyDF["date"].apply(lambda x: x.strftime('%M'))
asyDF["dtStr"] = asyDF["date"].apply(lambda x: x.strftime('%Y%m%d'))
asyDF = asyDF[ ["date", "AsyH", "AsyD", "SymH", "SymD"] ]
asyDF.head()

,date,AsyH,AsyD,SymH,SymD
0,2011-01-01 00:00:00,13.0,17.0,-10.0,1.0
1,2011-01-01 00:01:00,13.0,17.0,-10.0,1.0
2,2011-01-01 00:02:00,13.0,17.0,-10.0,1.0
3,2011-01-01 00:03:00,13.0,17.0,-10.0,1.0
4,2011-01-01 00:04:00,13.0,17.0,-10.0,1.0


In [5]:
velsDataDF = pandas.read_csv( "../data/processed-vels-geomag-fin.txt", sep=' ',\
                            infer_datetime_format=True,\
                            parse_dates=["date"])
# Get actual SAPS azimuths
velsDataDF["azim"] = -90.-velsDataDF["azim"]
velsDataDF.head()
# Filter out some values
# When there aren't good number of measurements at lats
# discard those values
cntMLTPntsDF = velsDataDF[ ["dst_bin", "normMLT", "MLAT"] \
                    ].groupby( ["dst_bin", "normMLT"] \
                    )["MLAT"].nunique().reset_index()
cntMLTPntsDF.columns = ["dst_bin", "normMLT", "MLATCntUnq"]
cntMLTPntsDF = cntMLTPntsDF[ cntMLTPntsDF["MLATCntUnq"] >=\
                    cutOffMLTMLATUniqCnt ][ ["dst_bin", "normMLT"]\
                        ].reset_index(drop=True)
velsDataDF = pandas.merge( velsDataDF, cntMLTPntsDF,\
                    on=["dst_bin", "normMLT"], how="inner" )
velsDataDF["hour"] = velsDataDF["date"].apply(lambda x: x.strftime('%H'))
velsDataDF["minute"] = velsDataDF["date"].apply(lambda x: x.strftime('%M'))
velsDataDF["dtStr"] = velsDataDF["date"].apply(lambda x: x.strftime('%Y%m%d'))
velsDataDF = velsDataDF [ ['azim',\
                      'MLAT', 'normMLT', 'vSaps',\
                      'date'] ]
velsDataDF.columns = ['azim',\
                      'MLAT', 'normMLT', 'vSaps',\
                      'date']
velsDataDF.head()

,azim,MLAT,normMLT,vSaps,date
0,-79.412573,62.0,-6.0,306.738497,2012-10-14 01:00:00
1,-76.475692,62.0,-6.0,301.958739,2012-10-14 01:02:00
2,-78.295769,62.0,-6.0,328.832509,2012-10-14 01:04:00
3,-78.597362,62.0,-6.0,357.951508,2012-10-14 01:06:00
4,-78.413884,62.0,-6.0,406.549966,2012-10-14 01:08:00


In [6]:
# Also merge with aymDF
# print "pre merge shape-->", velsDataDF.shape
velsDataDF = pandas.merge( velsDataDF, asyDF,\
                         on=["date"], how='inner')
velsDataDF.head()

,azim,MLAT,normMLT,vSaps,date,AsyH,AsyD,SymH,SymD
0,-79.412573,62.0,-6.0,306.738497,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0
1,-78.075135,61.5,-6.0,296.893207,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0
2,-82.467569,60.0,-6.0,220.239731,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0
3,-84.987310,60.5,-6.0,233.815511,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0
4,-79.086184,62.5,-6.0,343.746070,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0


In [10]:
asyHBins = [ 0, 30, 60, 90, 180 ]
velsDataDF = pandas.concat( [ velsDataDF, \
                    pandas.cut( velsDataDF["AsyH"], \
                               bins=asyHBins ) ], axis=1 )
velsDataDF.columns = ['azim','MLAT', 'normMLT', 'vSaps',\
                      'date', 'AsyH','AsyD', 'SymH', \
                      'SymD','AsyH_bin']
velsDataDF.head()

,azim,MLAT,normMLT,vSaps,date,AsyH,AsyD,SymH,SymD,AsyH_bin
0,-79.412573,62.0,-6.0,306.738497,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0,"(30, 60]"
1,-78.075135,61.5,-6.0,296.893207,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0,"(30, 60]"
2,-82.467569,60.0,-6.0,220.239731,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0,"(30, 60]"
3,-84.987310,60.5,-6.0,233.815511,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0,"(30, 60]"
4,-79.086184,62.5,-6.0,343.746070,2012-10-14 01:00:00,33.0,44.0,-48.0,2.0,"(30, 60]"


In [ ]:
sns.boxplot( x="normMLT", y="vSaps", hue="AsyH_bin",\
            data=velsDataDF, hue_order=asy_order,\
            palette=sns.color_palette(colors),\
            showfliers=False )

In [ ]:
asyDF = pandas.read_csv( "../data/Asy_processed.txt", sep=' ' )
asyDF["date"] = pandas.to_datetime(asyDF["datetimeStr"], format='%Y%m%d-%H-%M')
asyDF.head()

In [ ]:
velsDataDF = pandas.read_csv( "../data/processed-vels-geomag-fin.txt", sep=' ',\
                            infer_datetime_format=True,\
                            parse_dates=["date"])
velsDataDF.head()

In [ ]:
velsDataDF = pandas.merge( velsDataDF, asyDF,\
                         on=["date"], how='inner')